In [1]:
import pandas as pd
import numpy as np
import json
import datetime as dt
from datetime import datetime, date, time,timedelta
import pandas as pd
from csv import DictWriter
import urllib
import re,datetime
import os
import time
from os import walk
import re
import gc
from tqdm.notebook import tqdm, trange
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)


#############
#INPUTS
#############

delta = 15

backtest_start_date = '27-5-2016'
backtest_end_date = '31-1-2023'

index = "BANKNIFTY"
index_file_path = "D:\Correlation Backtest Data 2016-23\Correlation 6-16 to 1-23 data\Input Files\Index\BANKNIFTY.csv"

stfolder = "D:/Correlation Backtest Data 2016-23/Correlation 6-16 to 1-23 data/Input Files/Stocks"
    
wgt_lot_path = "D:\Correlation Backtest Data 2016-23\Correlation 6-16 to 1-23 data\Input Files\Others\BN Mcap & includeok - Copy - Copy.csv"

direc = r"D:/Correlation Backtest Data 2016-23/Correlation 6-16 to 1-23 data/Working Files - Only Calls"

output_path = r"D:/Correlation Backtest Data 2016-23/Correlation 6-16 to 1-23 data/Output Files - Only Calls"

index_lotsize = 1

initial_equity = 100000


#########################
#Creating Stock List
#########################

# filename = next(walk(stfolder), (None, None, []))[2]  # [] if no file
# stock_list_path = []
# for i in filename:
#     temp = stfolder +"/"+ i
#     stock_list_path.append(temp)

    
# stock_list = []
# for i in stock_list_path:
#     stock_list.append(i.replace(".csv","").replace(stfolder + "/",""))
# print(stock_list)

stock_list_df = pd.read_csv(wgt_lot_path, parse_dates = ['date'], dayfirst = True)
stock_list_df = stock_list_df[(stock_list_df['date'] >= pd.to_datetime(backtest_start_date, format = '%d-%m-%Y', dayfirst = True)) & (stock_list_df['includeok'] == 1)]
stock_list = stock_list_df['Security Symbol'].unique().tolist()
print(stock_list)

######################################    
#Creating weight and lot dictionary
######################################

"""wgtlot_df = pd.read_csv(wgt_lot_path)
wgtlot_dict = wgtlot_df.set_index('Symbol').T.to_dict('list')
print(wgtlot_dict)
symbols = list(wgtlot_dict.keys())"""

### Now adding Columns for weights below

######################################

lookback_period = 15
std_dev = 2

####################
#EXPIRY DATES
####################
symbols = stock_list.copy()
symbols.append(index)

exp_file_path = "D:\Correlation Backtest Data 2016-23\Correlation 6-16 to 1-23 data\Input Files\Others\MonthlyExpiry.csv"

exp_df = pd.read_csv(exp_file_path,parse_dates = ["curr_exp_date","Date"],dayfirst =True,usecols = ["curr_exp_date","Date","Weeks_to_Expiry", "Week of month"])
exp_df = exp_df[exp_df['Date'] >= pd.to_datetime(backtest_start_date, format = '%d-%m-%Y', dayfirst = True)]
#exp_df.rename({'curr_date': 'Date'}, axis=1, inplace=True)

['AXISBANK', 'BANKBARODA', 'BANKINDIA', 'CANBK', 'FEDERALBNK', 'HDFCBANK', 'ICICIBANK', 'INDUSINDBK', 'KOTAKBANK', 'PNB', 'SBIN', 'YESBANK', 'IDFCFIRSTB', 'RBLBANK', 'BANDHANBNK', 'AUBANK']


In [2]:
idx = pd.read_csv(index_file_path,parse_dates = ["Date"],dayfirst = True)
idx = idx[idx['Date'] >= pd.to_datetime(backtest_start_date, format = '%d-%m-%Y', dayfirst = True)]


df1 = idx[idx['Ticker'].str.endswith('-I')]
df2 = idx[idx['Ticker'].str.endswith('-II')]
df3 = idx[idx['Ticker'].str.endswith('-III')]

index_file_path = direc + "/BANKNIFTY.csv"

df1.to_csv(index_file_path)
df2.to_csv(index_file_path.replace("BANKNIFTY","BANKNIFTY-II"))
df3.to_csv(index_file_path.replace("BANKNIFTY","BANKNIFTY-III"))

print(idx.shape[0])
print(df1.shape[0]+df2.shape[0]+df3.shape[0])
    
for i in stock_list:
    df = pd.read_csv(stfolder + '/' + i + ".csv",parse_dates = ["Date"],dayfirst = True)
    df = df[df['Date'] >= pd.to_datetime(backtest_start_date, format = '%d-%m-%Y', dayfirst = True)]

    df1 = df[df['Ticker'].str.endswith('-I')]
    df2 = df[df['Ticker'].str.endswith('-II')]
    df3 = df[df['Ticker'].str.endswith('-III')]
    
    path = direc + '/' + i + ".csv"
    
    df1.to_csv(path)
    df2.to_csv(path.replace(i,i+"-II"))
    df3.to_csv(path.replace(i,i+"-III"))
    
    print(df.shape[0])
    print(df1.shape[0]+df2.shape[0]+df3.shape[0])

stfolder = direc
print(index_file_path)

297904
297904
93128
93128
75301
75301
24287
24287
64479
64479
72496
72496
96261
96261
116159
116159
81317
81317
76688
76688
72314
72314
132229
132229
55360
55360
49361
49361
53662
53662
37132
37132
18506
18506
D:/Correlation Backtest Data 2016-23/Correlation 6-16 to 1-23 data/Working Files - Only Calls/BANKNIFTY.csv


In [23]:
initial_equity = 1000000
target_percent = [1,1.5,2,2.5,3]
super_final = pd.DataFrame()
super_final_delta = pd.DataFrame()
initial_equity = 100000
leverage = 5

for t in tqdm(target_percent):
    target = (initial_equity*t)/100
    grp = exp_df.groupby("curr_exp_date")
    for name, group in tqdm(grp):
        #print(name)

        exx = grp.get_group(name)
        final1 = exx[['Date']].copy()
        final1['difference'] = 10000000
        final1['delta'] = 100

        date_min = exx['Date'].min()
        date_max = exx['Date'].max()
        delta_range = [15,20,25,30,35,40,45,50]
        count = 0

        weight = pd.read_csv(wgt_lot_path,parse_dates = ["date"],dayfirst = True,usecols = ["Security Symbol","Equal_Weightage","date","Lotsize"])
        weight = (weight[(weight['date'] >= pd.to_datetime(date_min, format = '%d-%m-%Y', dayfirst = True)) & (weight['date'] <= pd.to_datetime(date_max, format = '%d-%m-%Y', dayfirst = True))])
        #display(weight)
        wh_grp = weight.groupby("Security Symbol")
        exx[index + "_OG_Weight"] = 100
        exx[index + "_Lotsize"] = index_lotsize

        for i in stock_list:
            try:
                d1 = wh_grp.get_group(i)
            except:
                d1[i + "_OG_Weight"] = 0
                d1[i + "_Lotsize"] = d1["Lotsize"]
                d1['Date'] = d1["date"]
                exx = pd.merge(exx,d1[["Date",i + "_OG_Weight",i+ "_Lotsize"]],on= "Date", how='left')

                continue
            d1[i + "_OG_Weight"] = d1["Equal_Weightage"]
            d1[i + "_Lotsize"] = d1["Lotsize"]
            d1['Date'] = d1["date"]
            exx = pd.merge(exx,d1[["Date",i + "_OG_Weight",i+ "_Lotsize"]],on= "Date", how='left')
            exx['BANKNIFTY_Date_CE'] = exx['Date']
        
        exx.to_csv(output_path + '//' + 'exx_.csv')



        for delta in delta_range:

            #print(delta)

            idx = pd.read_csv(index_file_path,usecols = ["Date","Ticker","Option_Type","Strike",
                                                                                "OPT_Close","EQ_Close", "IV", 'Delta', f'Delta_{delta}_Strike'],parse_dates = ["Date"])

            idx = (idx[(idx['Date'] >= pd.to_datetime(date_min, format = '%d-%m-%Y', dayfirst = True)) & (idx['Date'] <= pd.to_datetime(date_max, format = '%d-%m-%Y', dayfirst = True))])
            idx = pd.merge(idx, exx, on = 'Date', how = 'left')
            #display(idx)

            idx_ce = idx[(idx["Strike"] == idx[f"Delta_{delta}_Strike"]) & (idx["Option_Type"] == "CE")].add_suffix("_CE")
            idx_pe = idx[(idx["Strike"] == idx[f"Delta_{delta}_Strike"]) & (idx["Option_Type"] == "PE")].add_suffix("_PE")

            final = idx_ce.merge(idx_pe, left_on='Date_CE', right_on='Date_PE').drop(columns = ["Option_Type_PE","Option_Type_CE"])
            #final = final.T.drop_duplicates().T  #Drop Duplicates Columns
            final = final.add_prefix(index +"_")
            final = final.sort_values(by=index +"_"+'Date_CE')
            #final.to_csv(output_path+"/check-1data.csv")

            for i in stock_list:
                #print(i)
                df1 = pd.read_csv(stfolder+ '/' + i + ".csv",parse_dates = ["Date"],usecols = ["Date","Ticker","Option_Type","Strike",
                                                                                "OPT_Close","EQ_Close","IV", 'Delta', f"Delta_{delta}_Strike"])

                df1 = (df1[(df1['Date'] >= pd.to_datetime(date_min, format = '%d-%m-%Y', dayfirst = True)) & (df1['Date'] <= pd.to_datetime(date_max, format = '%d-%m-%Y', dayfirst = True))])
                #df1 = pd.merge(df1, exx, on = 'Date', how = 'left')


                df_ce = df1[(df1["Strike"] == df1[f"Delta_{delta}_Strike"]) & (df1["Option_Type"] == "CE")].add_suffix("_CE")
                df_pe = df1[(df1["Strike"] == df1[f"Delta_{delta}_Strike"]) & (df1["Option_Type"] == "PE")].add_suffix("_PE")
                #print(df1.shape[0], df_ce.shape[0], df_pe.shape[0])
                temp = df_ce.merge(df_pe, left_on='Date_CE', right_on='Date_PE').drop(columns = ["Option_Type_PE","Option_Type_CE"])
                #temp = temp.T.drop_duplicates().T
                temp = temp.add_prefix(i +"_")
                final = final.merge(temp,how='left',left_on=index + '_Date_CE', right_on= i + '_Date_CE')

            #final = final.T.drop_duplicates().T
            #final = final.drop_duplicates()
            #display(final)

            #exx.rename(columns = {'Date':'BANKNIFTY_Date_CE'}, inplace = True)

            final = pd.merge(final, exx, on = 'BANKNIFTY_Date_CE', how = 'left')
            final.to_csv(output_path + '//' + 'exx1.csv')



            final['total_exposure'] = initial_equity*leverage*2

            final[index + '_exposure'] = final['total_exposure']/2
            final['stock_exposure'] = final['total_exposure']/2

            final[index + '_qty'] = (final[index + '_exposure'])/final[index+'_EQ_Close_CE']
            for i in stock_list:
                final[i+'_qty'] = (final['stock_exposure']*final[i+'_OG_Weight']/100)/final[i+'_EQ_Close_CE']
                
            
            filter_col = [col for col in final if (col.endswith('_qty') or col.endswith('_EQ_Close_CE'))]
            final[filter_col] = final[filter_col].replace(np.nan,0)
            
            for i in stock_list:
                if (final[i+'_qty'] == 0).all():
                    final[i+'_OPT_Close_CE'] = 0
            #final.to_csv(output_path + '//' + 'exx2.csv')

            final['stock_premium'] = 0
            
            for i in stock_list:
                final['stock_premium'] += final[i+'_OPT_Close_CE']*final[i+'_qty']

            final['index_premium'] = final['BANKNIFTY_OPT_Close_CE']*final[index + '_qty']

            final['total_premium'] = final['stock_premium'] - final['index_premium']

            #display(final)


            #final.to_csv(output_path + '//' + 'exx2.csv')

            final['target_premium'] = target 

            final['difference-check'] = final['target_premium'] - final['total_premium']
            final['Date'] = final['BANKNIFTY_Date_CE']

            if count > 0:
                final1 = final1.drop(['difference-check'], axis = 1)

            final1 = pd.merge(final1, final[['Date', 'difference-check']], on='Date',how='left' )

            final1['delta'] = np.where(abs(final1['difference-check']) < abs(final1['difference']), delta, final1['delta'])

            final1['difference'] = np.where(abs(final1['difference-check']) < abs(final1['difference']), final1['difference-check'], final1['difference'])


            count+=1



            #display(final)
            #final.to_csv(output_path + "/expcheck.csv")

        if os.path.exists(output_path + "/target_" + str(t) + "/super_final.csv"):
            final.to_csv(output_path + "/target_" + str(t) + "/super_final.csv", mode = 'a', header = False)
        else:
            final.to_csv(output_path + "/target_" + str(t) + "/super_final.csv")

        if os.path.exists(output_path + "/target_" + str(t) + "/super_final_delta.csv"):
            final1.to_csv(output_path + "/target_" + str(t) + "/super_final_delta.csv", mode = 'a', header = False)
        else:
            final1.to_csv(output_path + "/target_" + str(t) + "/super_final_delta.csv")

    #    super_final = super_final.append(final) 
    #    super_final_delta = super_final_delta.append(final1) 
    #    display(super_final)
    #    display(super_final_delta)

print("done")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
super_final.to_csv(output_path + '//' + 'super_final.csv')
super_final1.to_csv(output_path + '//' + 'super_final1.csv')

In [ ]:
data = pd.read_csv("D:\Correlation Backtest Data 2016-23\Correlation 6-16 to 1-23 data\Output Files - Only Calls\exx1.csv", parse_dates = ['BANKNIFTY_Date_CE'], dayfirst= True)
leverage =5
data['total_exposure'] = data['BANKNIFTY_EQ_Close_CE']*leverage*2

data['index_exposure'] = data['total_exposure']/2

for i in stock_list:
    data[i+'_exposure'] = (data['total_exposure']/2)*(data[i+'_OG_Weight']/100)
    if (data[i+'_OG_Weight'] == 0).all():
        data[i+'_quantity'] = 0
    else:
        data[i+'_quantity'] = data[i+'_exposure']/data[i+'_EQ_Close_CE']
    
display(data)
xxx




#ADDING QTYS
result = data.copy()
min_exp = []
for idi, row in result.iterrows():
    min_share_stock = row['Min_Share_Name'].replace("_Final_Weight","")
    #print(min_share_stock)
    value  = row[min_share_stock + "_EQ_Close_CE"]*row[min_share_stock + "_Lotsize"]
    min_exp.append(value)
    
result["MinExp"] = min_exp
result[index + "_exp"] = (result["MinExp"]/result["Min_Share_Value"])
#result[index + "_lot"] = (result[index + "_exp"]/(result[index + "_Lotsize"]*result[index + "_EQ_Close_CE"])).astype(int)
result[index + "_lot"] = (result[index + "_exp"]/(result[index + "_Lotsize"]*result[index + "_EQ_Close_CE"]))
result[index + "_qty"] = result[index + "_lot"]*result[index + "_Lotsize"]

result.to_csv(output_path + '//' + 'resultt.csv')

#for stocks with includeok=0, making '_Lotsize' & '_EQ_Close_CE' non-zero

filter_col_Lotsize = [col for col in result if col.endswith('_Lotsize')]
filter_col_Lotsize.remove(index + "_Lotsize")
result[filter_col_Lotsize] = result[filter_col_Lotsize].replace(0,1)


filter_col_EQ_Close = [col for col in result if col.endswith('_EQ_Close_CE')]
filter_col_EQ_Close.remove(index + "_EQ_Close_CE")
result[filter_col_EQ_Close] = result[filter_col_EQ_Close].replace(0,1)

result['index_exp'] = result[index + "_exp"]

result.to_csv(output_path + '//' + 'resulttcheckcheck.csv')

result['stock_exp'] = 0

for i in stock_list:
    print(i)
    if(i==min_share_stock):
        result[index + "_exp"]
        result[i + "_Final_Weight"]
        result[i + "_exp"] = result["MinExp"]
        result[i  + "_lot"] = 1
        result[i + "_qty"] = result[i + "_lot"]*result[i + "_Lotsize"]
        
        result['stock_exp'] += result[i + "_exp"]
        
    else:
        result[index + "_exp"]
        result[i + "_Final_Weight"]
        result[i + "_exp"] = (result[index + "_exp"]*result[i + "_Final_Weight"])
#         result[i  + "_lot"] = (result[i + "_exp"]/(result[i + "_Lotsize"]*result[i+ "_EQ_Close_CE"])).astype(int)
        result[i  + "_lot"] = (result[i + "_exp"]/(result[i + "_Lotsize"]*result[i+ "_EQ_Close_CE"]))
        result[i + "_qty"] = result[i + "_lot"]*result[i + "_Lotsize"]
        
        result['stock_exp'] += result[i + "_exp"]
        